## Facial Recognition Attendance System with Expression Analysis


## Installation of Packages


In [ ]:
pip install mediapipe

In [ ]:
pip install requests

In [ ]:
pip install deepface

# Importing the libraries

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from deepface import DeepFace
import csv
import datetime


## Downloading TensorFlow Lite Model (BlazeFace) using 'requests' library

In [ ]:
import requests

url = 'https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite'
filename = 'detector.tflite'

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
    print('Download completed.')
else:
    print('Failed to download the file.')


## Visualization utilities

To better demonstrate the Face Detector API, created a set of visualization tools that will be used in this colab. These will draw a bounding box around detected faces, as well as markers over certain detected points on the faces.

In [ ]:
from typing import Tuple, Union
import math
import cv2
import numpy as np

MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red


def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:
  """Converts normalized value pair to pixel coordinates."""

  # Checks if the float value is between 0 and 1.
  def is_valid_normalized_value(value: float) -> bool:
    return (value > 0 or math.isclose(0, value)) and (value < 1 or
                                                      math.isclose(1, value))

  if not (is_valid_normalized_value(normalized_x) and
          is_valid_normalized_value(normalized_y)):
    # TODO: Draw coordinates even if it's outside of the image bounds.
    return None
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px


def visualize(
    image,
    detection_result
) -> np.ndarray:
  """Draws bounding boxes and keypoints on the input image and return it.
  Args:
    image: The input RGB image.
    detection_result: The list of all "Detection" entities to be visualize.
  Returns:
    Image with bounding boxes.
  """
  annotated_image = image.copy()
  height, width, _ = image.shape

  for detection in detection_result.detections:
    # Draw bounding_box
    bbox = detection.bounding_box
    start_point = bbox.origin_x, bbox.origin_y
    end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
    cv2.rectangle(annotated_image, start_point, end_point, TEXT_COLOR, 3)

    # Draw keypoints
    for keypoint in detection.keypoints:
      keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y,
                                                     width, height)
      color, thickness, radius = (0, 255, 0), 2, 2
      cv2.circle(annotated_image, keypoint_px, thickness, color, radius)

    # Draw label and score
    category = detection.categories[0]
    category_name = category.category_name
    category_name = '' if category_name is None else category_name
    probability = round(category.score, 2)
    result_text = category_name + ' (' + str(probability) + ')'
    text_location = (MARGIN + bbox.origin_x,
                     MARGIN + ROW_SIZE + bbox.origin_y)
    cv2.putText(annotated_image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

  return annotated_image

CAPTURING THE IMAGE FROM CAMERA

In [ ]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    success,frame=cap.read()
    if not success:
        break
    cv2.imshow("Capture my Video",cv2.flip(frame,1))   
    if cv2.waitKey(100) & 0xFF == ord('e'):
        break
flipped_frame = cv2.flip(frame, 1)      
plt.imshow(flipped_frame[:, :, ::-1])
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Initialize the video capture object to use the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success,frame=cap.read()
    if not success:
        break
    flipped_frame = cv2.flip(frame, 1)
    org = (50, 50)
    cv2.putText(flipped_frame, "Present", org, cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=1.5, color=(255, 0, 0), thickness=3, lineType=cv2.LINE_AA)
    cv2.imshow("Attendance",flipped_frame)
    
    if cv2.waitKey(100) & 0xFF == ord('e'):
        break

cap.release()
cv2.destroyAllWindows()

OR INSERTING THE DOWNLOADED IMAGE FROM SYSTEM

In [ ]:
image_path = 'C:\\Users\\mnkmr\\Downloads\\SAMPLE FACE DETECT IMAGE.png'  
frame = cv2.imread(image_path)
if frame is None:
    print("Failed to load the image")
    exit()
cv2.imshow(" Download Image From System", frame)# Display the image
flipped_frame = frame     
plt.imshow(flipped_frame[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

## Running inference and visualizing the results

for downloaded image

In [ ]:
import cv2
import mediapipe as mp

# Load the image using OpenCV
image_path = 'C:\\Users\\mnkmr\\Downloads\\SAMPLE FACE DETECT IMAGE.png'
image = cv2.imread(image_path)

# Initialize MediaPipe face detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Convert the image to RGB format (required by MediaPipe)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run face detection on the image
with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    results = face_detection.process(image_rgb)
    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            ih, iw, _ = image.shape
            x, y, w, h = int(bbox.xmin * iw), int(bbox.ymin * ih), \
                         int(bbox.width * iw), int(bbox.height * ih)
            
            # Draw bounding box and landmarks on the image
            mp_drawing.draw_detection(image, detection) 

# Display the image with identification marks
cv2.imshow("Facial Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


for directly captured image

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# Create a MediaPipe face detection pipeline
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection()

# Open a video capture object
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run face detection
    results = face_detection.process(frame_rgb)

    # Draw bounding boxes and landmarks on the frame
    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            xmin = int(bbox.xmin * w)
            ymin = int(bbox.ymin * h)
            width = int(bbox.width * w)
            height = int(bbox.height * h)
            xmax = xmin + width
            ymax = ymin + height

            # Draw bounding box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Draw landmarks
            for keypoint in detection.location_data.relative_keypoints:
                x = int(keypoint.x * w)
                y = int(keypoint.y * h)
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    # Display the annotated frame
    cv2.imshow("Annotated Frame", frame)

    # Check for key press
    if cv2.waitKey(100) & 0xFF == ord('e'):
        break
# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()


# ATTENDANCE RECOGNITION


In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace
import csv
import datetime

In [ ]:
face_objs = DeepFace.extract_faces(flipped_frame, target_size=(224, 224), 
                                           detector_backend='opencv', enforce_detection=False,
                                           align=True, grayscale=False)
        

In [ ]:
if face_objs:
    extracted_face = face_objs[0]["face"]
    plt.imshow(extracted_face[:, :, ::-1])
    plt.axis('off')
    plt.show()
    obj=DeepFace.analyze(flipped_frame,enforce_detection=False)


In [ ]:
print(obj)

In [ ]:
analysis = DeepFace.analyze(flipped_frame, actions=['emotion'], enforce_detection=False)

        # If faces are detected, analysis will be a list
if isinstance(analysis, list):
    for obj in analysis:
       dominant_emotion = obj['dominant_emotion']
                
                # Display the detected emotion
print(f"Detected emotion: {dominant_emotion}")

In [ ]:
# Open a CSV file to record attendance
file = open('attendance.csv', mode='w', newline='')
writer = csv.writer(file)
writer.writerow(['Name', 'Date', 'Time', 'Expression'])

In [ ]:
name = "Student"  # Replace with actual name recognition logic if available
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
time = now.strftime("%H:%M:%S")
writer.writerow([name, date, time, dominant_emotion])

In [ ]:
            
# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
